In [1]:
from config.load_key import load_key
import os

load_key()
# 生产环境中请勿将 API Key 输出到日志中，避免泄露
print(f'''你配置的 API Key 是：{os.environ["DASHSCOPE_API_KEY"][:5]+"*"*5}''')

你配置的 API Key 是：sk-4b*****


我们在docs文件夹中准备了一些虚构的公司制度文件，接下来你将基于这些文件来创建RAG应用。

In [2]:
# 导入依赖
from llama_index.embeddings.dashscope import DashScopeEmbedding,DashScopeTextEmbeddingModels
from llama_index.core import SimpleDirectoryReader,VectorStoreIndex
from llama_index.llms.openai_like import OpenAILike

# 这两行代码是用于消除 WARNING 警告信息，避免干扰阅读学习，生产环境中建议根据需要来设置日志级别
import logging
logging.basicConfig(level=logging.ERROR)

print("正在解析文件...")
# LlamaIndex提供了SimpleDirectoryReader方法，可以直接将指定文件夹中的文件加载为document对象，对应着解析过程
documents = SimpleDirectoryReader('./docs').load_data()

print("正在创建索引...")
# from_documents方法包含切片与建立索引步骤
index = VectorStoreIndex.from_documents(
    documents,
    # 指定embedding 模型
    embed_model=DashScopeEmbedding(
        # 你也可以使用阿里云提供的其它embedding模型：https://help.aliyun.com/zh/model-studio/getting-started/models#3383780daf8hw
        model_name=DashScopeTextEmbeddingModels.TEXT_EMBEDDING_V2
    ))
print("正在创建提问引擎...")
query_engine = index.as_query_engine(
    # 设置为流式输出
    streaming=True,
    # 此处使用qwen-plus模型，你也可以使用阿里云提供的其它qwen的文本生成模型：https://help.aliyun.com/zh/model-studio/getting-started/models#9f8890ce29g5u
    llm=OpenAILike(
        model="qwen-plus-2025-07-28",
        api_base="https://dashscope.aliyuncs.com/compatible-mode/v1",
        api_key=os.getenv("DASHSCOPE_API_KEY"),
        is_chat_model=True
        ))
print("正在生成回复...")
streaming_response = query_engine.query('我们公司项目管理应该用什么工具')
print("回答是：")
# 采用流式输出
streaming_response.print_response_stream()

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


正在解析文件...
正在创建索引...
正在创建提问引擎...
正在生成回复...
回答是：
根据您提供的上下文信息，无法确定您公司项目管理应使用的具体工具。该文档主要涉及平安银行内部控制评价的相关内容，并未提及适用于项目管理的工具或建议。

### 2.3 保存与加载索引
你可能会发现，创建索引消耗的时间比较长。如果能够将索引保存到本地，并在需要使用的时候直接加载，而不是重新建立索引，那就可以大幅提升回复的速度，LlamaIndex提供了简单易实现的保存与加载索引的方法。

In [3]:
# 将索引保存为本地文件
index.storage_context.persist("knowledge_base/test")
print("索引文件保存到了knowledge_base/test")

索引文件保存到了knowledge_base/test


In [4]:
# 将本地索引文件加载为索引
from llama_index.core import StorageContext,load_index_from_storage
storage_context = StorageContext.from_defaults(persist_dir="knowledge_base/test")
index = load_index_from_storage(storage_context,embed_model=DashScopeEmbedding(
        model_name=DashScopeTextEmbeddingModels.TEXT_EMBEDDING_V2
    ))
print("成功从knowledge_base/test路径加载索引")

成功从knowledge_base/test路径加载索引


从本地加载索引后，你可以再次进行提问测试是否可以正常工作。

In [5]:
print("正在创建提问引擎...")
query_engine = index.as_query_engine(
    # 设置为流式输出
    streaming=True,
    # 此处使用qwen-plus模型，你也可以使用阿里云提供的其它qwen的文本生成模型：https://help.aliyun.com/zh/model-studio/getting-started/models#9f8890ce29g5u
    llm=OpenAILike(
        model="qwen-plus-2025-07-28",
        api_base="https://dashscope.aliyuncs.com/compatible-mode/v1",
        api_key=os.getenv("DASHSCOPE_API_KEY"),
        is_chat_model=True
        ))
print("正在生成回复...")
streaming_response = query_engine.query('我们公司项目管理应该用什么工具')
print("回答是：")
streaming_response.print_response_stream()

正在创建提问引擎...
正在生成回复...
回答是：
根据您提供的上下文信息，无法确定适用于您公司项目管理的具体工具。该文档主要涉及平安银行2023年度内部控制评价的相关内容，包括内部控制的评价依据、程序、方法及缺陷认定标准等，并未提及项目管理或推荐任何项目管理工具。

In [7]:
print("正在创建提问引擎...")
query_engine = index.as_query_engine(
    # 设置为流式输出
    streaming=True,
    # 此处使用qwen-plus-0919模型，你也可以使用阿里云提供的其它qwen的文本生成模型：https://help.aliyun.com/zh/model-studio/getting-started/models#9f8890ce29g5u
    llm=OpenAILike(
        model="qwen-plus-2025-07-28",
        api_base="https://dashscope.aliyuncs.com/compatible-mode/v1",
        api_key=os.getenv("DASHSCOPE_API_KEY"),
        is_chat_model=True
        ))
print("正在生成回复...")
streaming_response = query_engine.query('内部控制评价结论？')
print("回答是：")
streaming_response.print_response_stream()

正在创建提问引擎...
正在生成回复...
回答是：
内部控制评价结论为：于内部控制评价报告基准日，不存在财务报告内部控制重大缺陷，本行已按照企业内部控制规范体系和相关规定的要求在所有重大方面保持了有效的财务报告内部控制；同时，未发现非财务报告内部控制重大缺陷。自内部控制评价报告基准日至报告发出日之间，未发生影响内部控制有效性评价结论的因素。内部控制审计意见与本行对财务报告内部控制有效性的评价结论一致，且对非财务报告内部控制重大缺陷的披露与本行内部控制评价报告披露一致。

你可以将上述代码进行封装，以便在后续持续迭代时快速使用。